In [1]:
import numpy as np

# Total Return Swaps

In [2]:
def TRS(n_bonds, coupons, initial_prices, final_prices, fixed_rate, t, pay_total_return):
    
     initial_port_value = np.dot(n_bonds, initial_prices)
     final_port_value = np.dot(n_bonds, final_prices)
     coupon_pmt = np.sum(n_bonds * coupons/t * 100)
     total_return = (final_port_value - initial_port_value) + coupon_pmt
     rate = fixed_rate/t * initial_port_value

     return print(f'${(rate - total_return if pay_total_return else total_return - rate)[0]:,.2f}')

In [3]:
n_bonds = np.array([35_000, 45_000, 50_000])
coupons = np.array([0.06, 0.07, 0.08])
initial_prices = np.array([98.75, 101.5, 97.6])
final_prices = np.array([96.4, 97.3, 92.8])
fixed_rate = np.array([0.04])
t = np.array([2])
pay_total_return = True

TRS(n_bonds, coupons, initial_prices, final_prices, fixed_rate, t, pay_total_return)

$306,825.00


In [4]:
n_bonds = np.array([40_000, 50_000, 30_000])
coupons = np.array([0.0650, 0.0575, 0.0425])
initial_prices = np.array([106.83, 101.45, 98.36])
final_prices = np.array([104.27, 100.67, 97.24])
fixed_rate = np.array([0.0645])
t = np.array([4])
pay_total_return = False

TRS(n_bonds, coupons, initial_prices, final_prices, fixed_rate, t, pay_total_return)

$-204,531.06


In [5]:
n_bonds = np.array([300_000, 272_000])
coupons = np.array([0.0233, 0.0350])
initial_prices = np.array([109.88, 115.33])
final_prices = np.array([105.33, 106.87])
fixed_rate = np.array([0.0450])
t = np.array([2])
pay_total_return = True

TRS(n_bonds, coupons, initial_prices, final_prices, fixed_rate, t, pay_total_return)

$4,288,129.60


---

# Credit Default Swaps

El Spread de un CDS, es la tasa justa que se debería de pactar en un credit default swap en un momento dado en el timepo, en dunción de la probabilidad de default y la tasa de recuperación, y es calculado de la siguiente manera:

$$S_0 = \frac{\text{Protection Leg}}{RPV01}$$

donde el Protection Leg es calculado de la siguiente manera:

$$\text{Protection Leg} = \int_0^T \ Z(\tau) (1-R) dPD(\tau)$$

y RPV01 es calculado:

$$RPV01 = \sum_{j=1}^N Z(tj) \Delta (tj - 1, tj, B) Q (tj)$$

## **¿Qué es el RPV01? ¿Como se interpreta? ¿Que representan los elementos utilizados para calcularlos?**

**RPV01** (*Risky Present Value of 1 basis point*) representa el valor presente de 1 punto base durante la vida del CDS hasta vencimiento o default. (pp. 3) La fórmula para calcularlo se se ve de la siguiente manera:

$$RPV01 = \sum_{j=1}^N Z(tj) \Delta (tj - 1, tj, B) Q (tj)$$

Donde:

- $ Z(tj) $: factor de descuento libre de riesgo
- $ \Delta (tj - 1, tj, B) $: fracción del año entre pagos denotada como B
- $ Q (tj) $: probabilidad de supervivencia del emisor (pp. 7)

La interpretación del RPV01 sería la del valor presente esperado de los pagos de 1bp en el CDS, considerando la probabilidad de que el emisor **no haya hecho default**.

## **¿Qué es el Protection Leg? ¿Como se interpreta? ¿Que representan los elementos utilizados para calcularlos?**

El **Protection Leg** representa el valor presente esperado del pago que realiza el **vendedor de protección** al **comprador** en caso de default del emisor. Es decir que representa el LGD (1-RR), valor el cual deberá pagar el protection seller en caso de un evento de crédito. (pp. 8) Se calúcla con la fórmula:

$$\text{Protection Leg} = \int_0^T \ Z(\tau) (1-R) dPD(\tau)$$

Donde:

- $ Z(\tau) $: valor presente del pago por default en el tiempo $\tau$
- $ R $: tasa de Recovery Rate del subyacente
- $ (1 - R) $: pérdida esperada para el seller dada el default (LGD)
- $ dPD(\tau) $: diferencial de la función de probabilidad de default acumulada (incremento infitesimal de la probabilidad de que el emisor haga deafault en el timepo $\tau$)

El Protection Leg se interpreta como el valor presente esperado del LGD, es decir la pérdida esperada (ponderada) traída a valor presente.


## **¿Qué representa el spread de un CDS de manera matemática?, es decir, como se interpreta el ratio utilizado para calcularlo y que significa.**

El spread del CDS se calcula de la manera:

$$S_0 = \frac{\text{Protection Leg}}{RPV01}$$

Matemáticmanete significa que tanto representa el **Protection Leg** del **RPV01**. Es el pago justo (equilibrio) que habría en el CDS tomando en cuenta la pérdida esperada con base en el valor esperado del pago por el cambio de 1 basis. (pp. 11)

## **¿Qué otros elementos de un CDS es importante conocer y como se calculan?**

**MTM (Mark to Market)** (pp. 3-4)

Es el precio de mercado de un CDS ya existente cuando existen los cambios de spread. Esto se calcula de la siguiente manera:

$$MTM = (\text{Spread Nuevo} - \text{Spread Inicial}) \times RPV01$$

**Hazard Rates** (pp. 12-13)

Representa la probabilidad de que haya un evenbto de crédito por parte del emisor en un tiempo determinado. Este es calculado iterativamente mediante bootstrap de la forma:

$$\frac{S(t_v, t_v + 1Y)}{1- R} \ \sum_{n=3,6,9,12} \Delta(t_{n-3},t_n,B) Z(t_v,t_n)e^{-\lambda_{01}\tau_n} = \sum_{m=1}^{12} Z(t_v,t_m)(e^{-\lambda_{01}\tau_{m-1}}-e^{\lambda_{01}\tau_m})$$

## Referencias

Referencia principal (números de páginas):

- O’Kane, D., & Turnbull, S. (2003, abril). Valuation of credit default swaps. Lehman Brothers, Fixed Income Quantitative Credit Research.

Apoyo extra:

- Beumee, J., Brigo, D., Schiemert, D., & Stoyle, G. (2009, 7 de abril). Charting a course through the CDS Big Bang. Fitch Solutions, Quantitative Research, Global Special Report.
- Hull, J., & White, A. (2000). Valuing credit default swaps I: No counterparty default risk. Journal of Derivatives, 8(1), 29–40.
